In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ESAA-2/0913_train.csv')

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


In [4]:
max_len = 100
vocabulary = vectorizer.word_index

## (3) glove

* glove는 word2vec의 단점을 보완하기 위하여 등장하였습니다. 
* word2vec는 사용자가 지정한 주변 단어의 개수에 대해서만 학습이 이루어지기 때문에 데이터 전체에 대한 정보를 담기 어렵다는 단점을 지적하였습니다. (window 파라미터: 훈련 시 앞뒤로 고려하는 단어의 개수 default = 5)
* glove의 핵심 아이디어는 다음과 같습니다.

> 각 토큰들 간의 유사성은 그대로 가져가면서 데이터 전체에 대한 빈도를 반영하자


* glove embedding matrix를 keras의 embedding matrix에 주입하는 방법은 다음과 같습니다.
* Keras: 워드 임베딩을 수행하는 도구 Embedding()을 제공

1. 사전 훈련된 벡터를 갖고 있는 txt 파일을 다운로드 합니다.

In [5]:
import pandas as pd
glove_size = pd.read_table('/content/drive/MyDrive/ESAA-2/glove.txt',sep = " ")
glove_size.shape
# glove.txt: 하나의 줄당 101개의 값을 가짐.
## 단어 1개 + 임베딩 벡터의 100개의 차원에서의 각 값

(301506, 101)

2. txt 파일에 있는 단어와 벡터들을 glove dictionary에 저장합니다.

In [6]:
import numpy as np

# 훈련된 벡터를 저장할 빈 딕셔너리 생성
glove = dict()
# f: 사전 훈련된 벡터를 갖고 있는 txt파일
## open: 파일 생성을 위한 함수.(= txt 내용을 저장하기 위해 수행)
f = open('/content/drive/MyDrive/ESAA-2/glove.txt') 
for line in f:
    values = line.split() # 공백(스페이스, 탭, 엔터 등)을 기준으로 문자열을 나눈 값을 리스트로 저장
    word = values[0] # 임베딩 벡터가 의미하는 단어를 의미
    vector = np.asarray(values[1:], dtype='float32') # 임베딩 벡터의 100개의 차원에서의 각 값을 의미
    glove[word] = vector # 해당 key의 value로 저장
f.close() 
#열려 있는 파일 객체를 닫아 주는 역할
## (생략 가능, 이후에 파일을 다시 사용할 경우 오류가 발생할 수 있으므로, 닫아주는 편이 좋습니다.)

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장합니다.

In [7]:
# vocabulary: 문장으로부터 단어를 토큰화한 후, 숫자에 대응시킨 결과를 dict 형태로 반환한 것
## tokenizer.word_index
vocabulary

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'a': 5,
 'i': 6,
 'in': 7,
 'odin': 8,
 '”': 9,
 'he': 10,
 'was': 11,
 'it': 12,
 'you': 13,
 'that': 14,
 'his': 15,
 'with': 16,
 'had': 17,
 'for': 18,
 'as': 19,
 'her': 20,
 'at': 21,
 'not': 22,
 'my': 23,
 'is': 24,
 'but': 25,
 'have': 26,
 'be': 27,
 'she': 28,
 'me': 29,
 'him': 30,
 'said': 31,
 'on': 32,
 'all': 33,
 'so': 34,
 'this': 35,
 'from': 36,
 'by': 37,
 'which': 38,
 'were': 39,
 '’': 40,
 'there': 41,
 'no': 42,
 'one': 43,
 'what': 44,
 'been': 45,
 'would': 46,
 'we': 47,
 'they': 48,
 'are': 49,
 'your': 50,
 'if': 51,
 'an': 52,
 'very': 53,
 'mr': 54,
 'could': 55,
 'do': 56,
 'out': 57,
 'will': 58,
 'when': 59,
 'up': 60,
 'or': 61,
 'upon': 62,
 'them': 63,
 'more': 64,
 'man': 65,
 'now': 66,
 'who': 67,
 'some': 68,
 "'": 69,
 'am': 70,
 'know': 71,
 '“i': 72,
 'then': 73,
 'into': 74,
 'about': 75,
 'time': 76,
 'little': 77,
 'their': 78,
 'did': 79,
 'only': 80,
 'see': 81,
 'like': 82,
 'before': 83,
 'sh

In [8]:
# dict 형태의 자료에 enumerate를 적용할 경우
a = {'hello':1, 'world':2, 'Esaa':5, 'statistics':10}
for index,word in enumerate(a):
  print(index,'/',word)

0 / hello
1 / world
2 / Esaa
3 / statistics


In [9]:
# glove.txt: 하나의 줄당 101개의 값을 가지는 리스트
## 단어 1개 + 임베딩 벡터의 100개의 차원에서의 각 값
## glove의 경우, 자료가 txt 형태로 존재 => embedding_matrix의 열의 수 주의
embedding_matrix = np.zeros((vocab_size, 100)) #100차원의 임베딩 매트릭스 생성 
## k번째 행의 값 = vocabulary의 k번째 단어에 대한 100차원의 각 값

for index, word in enumerate(vocabulary): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 glove에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_matrix에 저장합니다.
    else: # 넘겨 받은 토큰이 glove에 존재하지 않을 경우 = 훈련되지 않은 토큰
        print("'"+word+"'는 glove에 없는 단어입니다.")
        break

'odin'는 glove에 없는 단어입니다.


=> 훈련되지 않은 토큰이 1개 존재 

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [10]:
from keras.models import Sequential
from keras.layers import Embedding
#순차 모델 생성
model = Sequential()
#순차적으로, 워드 임베딩 결과를 layer로 더함.
model.add(Embedding(vocab_size, 100,weights = [embedding_matrix], input_length = max_len))

## 4. FastText

* FastText의 핵심 아이디어는 단어 단위가 아닌 sub 단어를 단위로 사용합니다. 즉 다음과 같습니다.

> word2vec -> "apple" 학습


> FastText -> "ap", "pp", "pl", "le" 학습

* 따라서 미리 학습되지 않은 단어들에 대한 vector도 표현해준다는 장점이 있습니다. 이용은 다음과 같습니다.

1. 사전 훈련된 bin 파일을 다운로드 합니다.

2. vec 파일을 gensim을 활용하여 읽어옵니다.

In [11]:
from gensim.models.keyedvectors import KeyedVectors
# 단어 포함 파일에 오류가 있을 수 있으므로, 매개 변수에 unicode_errors='ignore'를 사용하는 것이 좋습니다.
FastText = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/ESAA-2/fasttext.vec', binary = True, unicode_errors='ignore')

In [12]:
from torchtext.vocab import Vectors
FastText_vectors = Vectors('/content/drive/MyDrive/ESAA-2/fasttext.vec')
print('한 단어를 표현하는 차원 수:', FastText_vectors.dim)
print('단어 수: ', len(FastText_vectors.itos))

한 단어를 표현하는 차원 수: 100
단어 수:  358043


3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장합니다.

In [13]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vocabulary): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in FastText: #넘겨 받은 토큰이 FastText에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = FastText[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else: # 넘겨 받은 토큰이 FastText에 존재하지 않을 경우 = 훈련되지 않은 토큰
        print("'"+word+"'는 FastText에 없는 단어입니다.")
        break

'the'는 FastText에 없는 단어입니다.


=> 훈련되지 않은 토큰이 1개 존재

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [14]:
from keras.models import Sequential
from keras.layers import Embedding
#순차 모델 생성
model = Sequential() 
#순차적으로, 워드 임베딩 결과를 layer로 더함.
model.add(Embedding(vocab_size,300,weights = [embedding_matrix], input_length = max_len))

=> 이렇게 word embedding에 대해서 알아보았습니다. 

=> 본 발표에서는 다루지 않았지만, doc2vec으로도 word embedding을 진행할 수 있습니다.

*  doc2vec는 word2vec에서 확장된 개념이라고 할 수 있습니다.
*  즉, 단어 단위가 아닌 문장 단위의 벡터를 특정 차원으로 분석하는 것입니다.
* 예측하는 방법에는 PV-DM 과 PV-DBOW 두 방식이 있습니다.
*  이번 발표에서 다루지는 않았지만, 추후에 공부해보시는 것을 추천드립니다.